In [9]:
!pip install -U transformers datasets accelerate
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments
from datasets import load_dataset, Dataset
import pandas as pd
import re

In [2]:
from datasets import load_dataset

# 1️⃣ 데이터 불러오기
ds = load_dataset("ChuGyouk/PubMedQA-test-Ko")

# 2️⃣ test split을 CSV로 저장
ds["test"].to_csv("PubMedQA_test.csv", index=False)

print("✅ 저장 완료! 파일명: PubMedQA_test.csv")


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/155 [00:00<?, ?B/s]

pubmedQA_test_translated.json: 0.00B [00:00, ?B/s]

Generating test split:   0%|          | 0/500 [00:00<?, ? examples/s]

Creating CSV from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

✅ 저장 완료! 파일명: PubMedQA_test.csv


In [3]:
import pandas as pd
import re
df = pd.read_csv("PubMedQA_test.csv")


In [12]:
def clean_text(text):
    """텍스트 내 불필요한 특수문자, 공백, 영어 혼용 제거"""
    text = re.sub(r'\s+', ' ', str(text))   # 다중 공백 제거
    text = re.sub(r'[^가-힣0-9a-zA-Z .,?!]', '', text)  # 특수문자 제거
    return text.strip()
sample_q = clean_text(df.iloc[0]['QUESTION'])
sample_a = clean_text(df.iloc[0]['LONG_ANSWER'])
print("🧹 전처리 예시:")
print("Q:", sample_q)
print("A:", sample_a)

🧹 전처리 예시:
Q: 항문 직장 내 초음파 검사는 배변 장애에 유용합니까?
A: 선형 항문 직장 내시경 검사는 배변 운동 중 항문 괄약근과 항문 직장근의 이완이 불완전하거나 심지어 부재 한 것으로 나타났습니다. 이 연구는 골반저근긴장이상증 또는 이완불능증의 진단에 있어 이 우아한 초음파 촬영 기법의 가치를 강조합니다.


In [13]:
from datasets import Dataset

# "질문 + 답변" 형식으로 변환
train_texts = [
    f"질문: {q}\n답변: {a}"
    for q, a in zip(df["QUESTION"], df["LONG_ANSWER"])
]

dataset = Dataset.from_dict({"text": train_texts})


In [4]:
!pip install -U transformers datasets accelerate

from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, Trainer, TrainingArguments
from datasets import Dataset
import pandas as pd
import re
import torch

# ✅ GPU 디버깅 옵션 (정확한 에러 위치 보기)
%env CUDA_LAUNCH_BLOCKING=1

# ✅ 데이터 로드
df = pd.read_csv("PubMedQA_test.csv").dropna(subset=["QUESTION", "LONG_ANSWER"])

# ✅ 전처리
def clean_text(x):
    x = re.sub(r"\s+", " ", str(x))
    return x.strip()

df["QUESTION"] = df["QUESTION"].apply(clean_text)
df["LONG_ANSWER"] = df["LONG_ANSWER"].apply(clean_text)
df = df[df["QUESTION"].str.len() > 5]
df = df[df["LONG_ANSWER"].str.len() > 5]

df["input"] = "질문: " + df["QUESTION"]
df["output"] = df["LONG_ANSWER"]
dataset = Dataset.from_pandas(df[["input", "output"]])

# ✅ 모델 로드
model_name = "paust/pko-t5-small"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

# ✅ pad_token 설정
tokenizer.pad_token = tokenizer.eos_token
model.config.pad_token_id = tokenizer.pad_token_id

# ✅ 토크나이징 + 안전한 라벨 처리
def tokenize_fn(batch):
    inputs = tokenizer(batch["input"], truncation=True, padding="max_length", max_length=256)
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(batch["output"], truncation=True, padding="max_length", max_length=256)
    # pad 토큰을 -100으로 교체 (loss 무시)
    inputs["labels"] = [
        [tok if tok != tokenizer.pad_token_id else -100 for tok in seq]
        for seq in labels["input_ids"]
    ]
    return inputs

tokenized = dataset.map(tokenize_fn, batched=True)
tokenized.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])

# ✅ sanity check (문제 있는 샘플 탐색)
for i in range(3):
    bad = [tok for tok in tokenized[i]["labels"] if tok < -1000 or tok > tokenizer.vocab_size]
    if bad:
        print("⚠️ 잘못된 토큰 발견:", bad, "at index", i)

print("✅ 데이터 확인 완료, GPU 초기화 중...")

torch.cuda.empty_cache()

# ✅ 학습 설정
training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=2,
    num_train_epochs=2,
    learning_rate=5e-5,
    fp16=False,
    logging_steps=50,
    report_to="none"
)

# ✅ Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized,
)

# ✅ 학습
trainer.train()


env: CUDA_LAUNCH_BLOCKING=1


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/726 [00:00<?, ?B/s]

...

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

/usr/local/lib/python3.12/dist-packages/transformers/tokenization_utils_base.py:4034: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


✅ 데이터 확인 완료, GPU 초기화 중...


Step,Training Loss
50,3.074900
100,2.919200
150,2.725500
200,2.711400
250,2.645600
300,2.558100
350,2.531800
400,2.496600
450,2.503700
500,2.547600


TrainOutput(global_step=500, training_loss=2.6714305267333986, metrics={'train_runtime': 136.8811, 'train_samples_per_second': 7.306, 'train_steps_per_second': 3.653, 'total_flos': 107281907712000.0, 'train_loss': 2.6714305267333986, 'epoch': 2.0})